In [1]:
import pandas as pd
from decimal import Decimal
import json
import numpy as np
from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )

Carbon Version v2.0-beta3 (12/Dec/2022)


# Carbon Simulation - Demo-5-6 Simulator Import_Export

Here we demonstrate the preferred setup to import and export json test files from the simulator

In [ ]:
def load_json(filename):     
    '''
    :filename:  the convention is to specify the trade by source or trade by target in the filename i.e. tradeByTargetAmount.json
                this is necessary as the `is_by_target` field specifies the trade direction for the actionList
    '''   
    if ('Target' in filename) or ('target' in filename):
        is_by_target = True
    else:
        is_by_target = False
    f = open(filename , "r")
    bm = f.read()
    loaded_json = json.loads(bm)
    f.close()
    return(loaded_json, is_by_target)

def decimalize_dict(dicta):
    return({k:Decimal(v) for k,v in dicta.items()})

def calc_y_int(liquidity, highestRate, lowestRate, marginalRate):
    return(liquidity * (highestRate.sqrt() - lowestRate.sqrt()) / (marginalRate.sqrt() - lowestRate.sqrt()))

def parse_order(order0, order_count):
    order0 = decimalize_dict(order0)
    order0['delta_liquidity'] = order0['newLiquidity'] - order0['liquidity']
    order0['order'] = order_count
    order0_df = pd.DataFrame.from_dict(order0, 'index', columns=[f'{order_count}'])
    return(order0_df)

def parse_strategy(strategy, strat_count, order_count):
    orders = pd.DataFrame()
    for order in strategy:
        order_df = parse_order(order, order_count)
        orders = pd.concat([orders,order_df.T])
        order_count += 1
    orders['strategy'] = strat_count
    strat_count += 1
    return(orders, strat_count, order_count)

def format_json_as_df(json, scenario_index = 0):
    '''
    :json:             the loaded json file
    :scenario_index:   each json file has multiple scenarios, make a selection
    '''
    actionList = json[scenario_index]['actions']
    
    print(f'The number of scenarios is {len(json)}')
    print(f'You selected scenario: {scenario_index}')  
    print(f'The number of actions in this scenario are: {len(actionList)}')

    if scenario_index >= len(json):
        print('Please selected a valid scenario_index')
        return()
    else:
        strats = pd.DataFrame()
        strat_count = 0
        order_count = 0
        for strategy in json[scenario_index]['strategies']:
            orders, strat_count, order_count = parse_strategy(strategy, strat_count, order_count)
            strats = pd.concat([strats, orders])
        return(strats, actionList)

def parse_actions(Sim, actionList, strats, is_by_target):
    '''
    parse_actions processes trades from the actionList

    :Sim:           the specific simulator environment to trade against
    :actionList:    the list of actions provides from the test json
    :strats:        the preprocessed dataframe of strategies from the test json
    :is_by_target:  the specification of trading by target or by source
    '''

    # print(f'is_by_target {is_by_target}')
    for action in actionList:
        print(action)
        targetOrder = strats.query(f"strategy=={action['strategyId']}")
        targetOrderId = targetOrder['order'].values[0]
        targetOrderToken = Sim.state()['orders'].query(f"id=={targetOrderId}")['tkn'].values[0]
        tokenAmount = Decimal(action['tokenAmount'])
        if not is_by_target:
            if action['orderIndex'] == 1:
                Sim.amm_buys(Sim.carbon_pair.other(targetOrderToken),tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_source
            else:    
                targetOrderId += 1
                Sim.amm_buys(targetOrderToken,tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_source
            print(targetOrderToken, targetOrderId, tokenAmount)
        else:
            if action['orderIndex'] == 1:
                Sim.amm_sells(targetOrderToken,tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_target
            else:
                targetOrderId += 1
                Sim.amm_sells(Sim.carbon_pair.other(targetOrderToken),tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_target
            print(Sim.carbon_pair.other(targetOrderToken), targetOrderId, tokenAmount)
        
def compare_results(strats, initial_orders, final_orders):
    '''
    :strats:            the preprocessed dataframe of strategies from the test json
    :initial_orders:    the state of the simulator orders immediately after importing
    :final_orders:      the state of the simulator orders immediately after performing trade actions
    '''

    df = pd.concat([strats[['newLiquidity', 'newMarginalRate']].reset_index(drop=True), final_orders[['tkn', 'y', 'p_marg']].reset_index(drop=True)], axis=1)
    df.columns = ['json_liquidity', 'json_price', 'tkn', 'sim_liquidity', 'sim_price']
    df = pd.concat([df.reset_index(drop=True), strats[['delta_liquidity']].reset_index(drop=True), (final_orders['y'] - initial_orders['y']).reset_index(drop=True)], axis=1)

    df = df[['tkn','json_price', 'sim_price', 'json_liquidity', 'sim_liquidity', 'delta_liquidity', 'y']]
    # Remember that the prices for one side of the strategy are flipped when imported to the simulator so they need to be inverted for comparison here
    df.loc[:,'sim_price'] = [1/df.sim_price[i] if i%2==1 else df.sim_price[i] for i in df.index]
    df.columns = ['tkn','json_price', 'sim_price', 'json_liquidity', 'sim_liquidity', 'delta_json_liquidity', 'delta_sim_liquidity']
    return(df)

def check_results(results, tolerance, short_long = 'short'):
    '''
    :results:       a df output from compare_results
    :tolerance:     the absolute tolerance acceptable between compared values, default 1 wei
    :short_long:    "short" returns everything in the table passed
                    "long" returns itemized list of what passed
    '''
    test_marg_rates = np.isclose(results.json_price.astype(float), results.sim_price.astype(float), rtol=tolerance)
    test_final_liq = np.isclose(results.json_liquidity.astype(float), results.sim_liquidity.astype(float), rtol=tolerance)
    test_delta_liq = np.isclose(results.delta_json_liquidity.astype(float), results.delta_sim_liquidity.astype(float), rtol=tolerance)

    if short_long == 'short':
        print('Short check')
        print('Testing marginal rate:', test_marg_rates.all())
        assert test_marg_rates.all()
        print('Testing final liquidity:', test_final_liq.all())
        assert test_final_liq.all()
        print('Testing delta liquidity:', test_delta_liq.all())
        assert test_delta_liq.all()
    
    else:
        return(pd.DataFrame(zip(test_marg_rates, test_final_liq, test_delta_liq), columns=['newMarginalRate', 'newLiquididy', 'deltaLiquididty'], index=results.index))


def test_all_scenarios(filename, tolerance):

    '''
    :tolerance: the absolute acceptable tolerance
    '''

    # load the json
    bm_json, is_by_target = load_json(filename)

    # loop over each scenario
    for scenario_index in range(len(bm_json)):
        print('\n')
        # read in strategies and actions
        strats, actionList = format_json_as_df(bm_json, scenario_index = scenario_index)

        # initialize simulator
        FastSim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='fast', raiseonerror=True)

        # add strategies to simulator
        for i in strats.order[::2]:
            FastSim.add_strategy(
                tkn='USDC',
                amt_sell=strats.liquidity[i],
                psell_start=strats.highestRate[i],
                psell_end=strats.lowestRate[i],
                amt_buy=strats.liquidity[i+1],
                pbuy_start=1/strats.highestRate[i+1],
                pbuy_end=1/strats.lowestRate[i+1],
                psell_marginal=strats.marginalRate[i],
                pbuy_marginal=1/strats.marginalRate[i+1],
            )
        # snapshot the orders on the simulator
        initial_orders = FastSim.state()['orders']

        # process the trades from the action list
        parse_actions(FastSim, actionList, strats, is_by_target)

        # snapshot the orders on the simulator post-trade
        final_orders = FastSim.state()['orders']
        
        # generate a comparison report
        results = compare_results(strats, initial_orders, final_orders)

        # check that the difference between simulator and json in within 1 wei
        check_results(results, tolerance = tolerance, short_long = 'short')
    
    print('All tests complete')

# Simply run the whole check on every scenario
Using test_all_scenarios or follow the notebook for step-by-step processing

In [ ]:
filename='20221219_tradeByTargetAmount.json'
test_all_scenarios(filename, tolerance= 0.1)

# Import

First we load the json test file and return a df of the strategies and a list of actions

** If you want to use a specific scenario you need to specify the scenario_index here **

In [ ]:
# Specify the scenario_index for a different scenario
bm_json, is_by_target = load_json(filename)
strats, actionList = format_json_as_df(bm_json, scenario_index = 0)
strats

# Initialize the Simulator and create the orders
Here the orders are entered into the simulator and the prices for one side of the strategy are flipped to coincide with the price convention

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

FastSim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='fast', raiseonerror=True)

for i in strats.order[::2]:
    FastSim.add_strategy(
        tkn='USDC',
        amt_sell=Decimal(str(strats.liquidity[i])),
        psell_start=strats.highestRate[i],
        psell_end=strats.lowestRate[i],
        amt_buy=strats.liquidity[i+1],
        pbuy_start=1/strats.highestRate[i+1],
        pbuy_end=1/strats.lowestRate[i+1],
        psell_marginal=strats.marginalRate[i],
        pbuy_marginal=1/strats.marginalRate[i+1],
    )
initial_orders = FastSim.state()['orders']
initial_orders

# Parse the actions to perform the trades

In [ ]:
parse_actions(FastSim, actionList, strats, is_by_target)

FastSim.state()['trades']

# Display the status of the orders post-trade

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

final_orders = FastSim.state()['orders']
final_orders

# Compare the results

We can generate the comparison df using compare_results then run a check over this to see if it meets a defined tolerance

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

results = compare_results(strats, initial_orders, final_orders)
results

In [ ]:
check_results(results, tolerance = 1e-1, short_long = 'short')

In [ ]:
check_results(results, tolerance = 1e-1, short_long = 'long')

# Export (WIP still a few bugs)

In [ ]:
def generate_json_actions(trade_a):

    json_actions = {}
    sub_actions_list = []

    if trade_a['success'] & trade_a['trades'].exec.all():
        subdf = trade_a['trades'].query('subid!="A"').copy()
        subdf.reset_index(inplace=True, drop=True)
        trade_is_by_target = trade_a['is_by_target']
        tkn = trade_a['tkn']
        if trade_is_by_target & (FastSim.carbon_pair.basetoken==tkn):
            orderIndex = 0
        elif trade_is_by_target & (FastSim.carbon_pair.basetoken!=tkn):
            orderIndex = 1
        elif (not trade_is_by_target) & (FastSim.carbon_pair.basetoken==tkn):
            orderIndex = 0
        elif (not trade_is_by_target) & (FastSim.carbon_pair.basetoken!=tkn):
            orderIndex = 1
        else:
            print('Failed')
        print(f'trade_is_by_target: {trade_is_by_target}')

        for i in subdf.index:
            sub_actions = {}
            sub_actions['strategyId'] = subdf.routeix[i]//2
            sub_actions['orderIndex'] = orderIndex
            if trade_is_by_target:
                sub_actions['tokenAmount'] = subdf.amt1[i]
            else:
                sub_actions['tokenAmount'] = subdf.amt2[i]
            sub_actions_list += [sub_actions]
            
            
    json_actions['actions'] = sub_actions_list
    return(json_actions, trade_is_by_target)

def generate_json_strategies(initial_orders, final_orders):
    json_strats = {}
    sub_dict_list = []
    for i in initial_orders.index[::2]:
        sub_dict0 = {}
        sub_dict0['liquidity'] = str(initial_orders.y[i+1])
        sub_dict0['lowestRate'] = str(initial_orders.p_end[i+1] - 0.000000001)
        sub_dict0['highestRate'] = str(initial_orders.p_start[i+1] + 0.000000001)
        sub_dict0['marginalRate'] = str(initial_orders.p_marg[i+1])
        sub_dict0['newLiquidity'] = str(final_orders.y[i+1])
        sub_dict0['newMarginalRate'] = str(final_orders.p_marg[i+1])
        
        sub_dict1 = {}
        sub_dict1['liquidity'] = str(initial_orders.y[i])
        sub_dict1['lowestRate'] = str(1/(initial_orders.p_end[i] + 0.000000001))
        sub_dict1['highestRate'] = str(1/(initial_orders.p_start[i] - 0.000000001))
        sub_dict1['marginalRate'] = str(1/initial_orders.p_marg[i])
        sub_dict1['newLiquidity'] = str(final_orders.y[i])
        sub_dict1['newMarginalRate'] = str(1/final_orders.p_marg[i])
        sub_dict_list += [[sub_dict0,sub_dict1]]
    json_strats['strategies'] = sub_dict_list
    return(json_strats)


def format_simulator_json(json_strats, json_actions):
    json_simulator = []
    simulator_setup = {**json_strats, **json_actions}
    json_simulator += [simulator_setup]
    return(json_simulator)

# Initialize a simulator and add some strategies

In [ ]:
# New Sim
FastSim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='fast', raiseonerror=True)

In [ ]:
# Add strategies
FastSim.add_strategy("ETH", 10, 2000, 2500, 10000, 1020, 1020)
FastSim.add_strategy("ETH", 10, 2010, 2810, 10000, 1210, 1210)
FastSim.add_strategy("ETH", 10, 2020, 2220, 10000, 1050, 1050)
FastSim.add_strategy("ETH", 10, 2030, 2060, 10000, 1055, 1055)
FastSim.add_strategy("ETH", 10, 2040, 2080, 10000, 1240, 1040)

# Snapshot the simulator orders
initial_orders = FastSim.state()['orders']

# Do a trade and process it into an actionList

In [ ]:
# Do a trade
# trade_a = FastSim.amm_sells('ETH', 30, execute=True) # route_trade_by_target
# trade_a = FastSim.amm_buys('ETH', 10, execute=True) # route_trade_by_source
# trade_a = FastSim.amm_sells('USDC', 31000, execute=True) # route_trade_by_target
trade_a = FastSim.amm_buys('USDC', 42000, execute=True) # route_trade_by_source

# Generate the actions component of the json
json_actions, trade_is_by_target  = generate_json_actions(trade_a)

In [ ]:
trade_a['trades']

In [ ]:
json_actions

In [ ]:
# Snapshot the orders
final_orders = FastSim.state()['orders']

# Generate the strategies component of the json

In [ ]:
# Generate the strategies component of the json
json_strats = generate_json_strategies(initial_orders, final_orders)

# Combine and format the jsons

In [ ]:
json_simulator = format_simulator_json(json_strats, json_actions)
json_simulator

# Optionally Save

In [ ]:
# from datetime import datetime
# str_date = datetime.today().strftime("%Y%m%d")
# unique_id = datetime.now().strftime("%f")

# if trade_is_by_target:
#     str_type = "tradeByTarget"
# else:
#     str_type = "tradeBySource"
    
# export_filename = f"{str_date}_{str_type}_test_{unique_id}.json"

# out_file = open(export_filename, "w")
# json.dump(json_simulator, out_file, indent=4)
# out_file.close()

# print(f'Saved as {export_filename}')